In [1]:
import numpy as np
import pandas as pd
import random

#### Load the matrices

In [2]:
battery_expenditure = pd.read_csv('battery_expenditure_matrix.csv')
distance_matrix = pd.read_csv('distance_matrix.csv')
time_matrix = pd.read_csv('time_matrix.csv')

In [3]:
customer_demands = [[3,4],[5,15],[3,7],[8,10],[6,13]]

#### Generating random combination

In [4]:
num_sequences=20
start_node=0
end_node=16
battery_threshold=10

w1=0.5
w2=0.5

In [5]:
def sequenceIsValid(route):
    # check for validity if car strength not increase 3

    # check for validity if every customer reaches destination

    #check if battery level not decrease less than 0
    for i in range(len(route) - 1):
        node1 = route[i]
        node2 = route[i + 1]
                
        if node1 == 1 or node1 == 2:
            battery_level=78

            battery_consumption = battery_expenditure.iloc[node1, node2]
            battery_level -= battery_consumption
            if battery_level < battery_threshold:
                return False
    return True

In [6]:
def generate_random_sequences(num_sequences, battery_threshold,customer_demands=customer_demands):
    sequences = []

    for _ in range(num_sequences):
        while True:

            route = [0]  # Start at node 0
            # Randomly shuffle the customer demands
            shuffled_demands = random.sample(customer_demands, len(customer_demands))
            for demand in shuffled_demands:
                route.append(demand[0])  # Start node of the customer demand
                route.append(demand[1])  # End node of the customer demand

            route.append(end_node)  # End at node 0
            
            valid_sequence = sequenceIsValid(route)
            
            if valid_sequence:
                sequences.append(route)
                break

    return sequences

In [7]:
random_seq=generate_random_sequences(num_sequences,10)
random_seq

[[0, 6, 13, 3, 4, 5, 15, 8, 10, 3, 7, 16],
 [0, 3, 4, 6, 13, 3, 7, 8, 10, 5, 15, 16],
 [0, 3, 7, 6, 13, 8, 10, 3, 4, 5, 15, 16],
 [0, 5, 15, 3, 4, 8, 10, 3, 7, 6, 13, 16],
 [0, 3, 7, 5, 15, 3, 4, 8, 10, 6, 13, 16],
 [0, 3, 7, 8, 10, 3, 4, 6, 13, 5, 15, 16],
 [0, 3, 4, 8, 10, 5, 15, 3, 7, 6, 13, 16],
 [0, 8, 10, 3, 7, 6, 13, 3, 4, 5, 15, 16],
 [0, 8, 10, 6, 13, 5, 15, 3, 7, 3, 4, 16],
 [0, 5, 15, 8, 10, 3, 4, 3, 7, 6, 13, 16],
 [0, 5, 15, 6, 13, 3, 4, 3, 7, 8, 10, 16],
 [0, 8, 10, 3, 7, 5, 15, 3, 4, 6, 13, 16],
 [0, 8, 10, 3, 4, 5, 15, 6, 13, 3, 7, 16],
 [0, 3, 7, 6, 13, 5, 15, 3, 4, 8, 10, 16],
 [0, 8, 10, 5, 15, 6, 13, 3, 7, 3, 4, 16],
 [0, 5, 15, 3, 7, 6, 13, 8, 10, 3, 4, 16],
 [0, 8, 10, 3, 4, 3, 7, 6, 13, 5, 15, 16],
 [0, 3, 4, 8, 10, 5, 15, 3, 7, 6, 13, 16],
 [0, 8, 10, 5, 15, 3, 7, 3, 4, 6, 13, 16],
 [0, 5, 15, 3, 4, 3, 7, 8, 10, 6, 13, 16]]

In [8]:
def calculate_cost(sequence, time_matrix, distance_matrix, w1, w2):
    cost1 = 0
    cost2 = 0
    cost = 0

    for i in range(len(sequence) - 1):
        node1 = sequence[i]
        node2 = sequence[i + 1]

        cost1 += time_matrix.iloc[node1, node2]
        cost2 += distance_matrix.iloc[node1, node2]

        cost += (w1 * time_matrix.iloc[node1, node2]) + (w2 * distance_matrix.iloc[node1, node2])

    return pd.DataFrame({'Σt': [cost1], 'Σd': [cost2], 'w1*Σt+w2*Σd': [cost]})

In [9]:
cost_for_each_sequence = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2) for sequence in random_seq]
result_df = pd.concat(cost_for_each_sequence, ignore_index=True)

In [10]:
result_df

,Σt,Σd,w1*Σt+w2*Σd
0,3.899556,185.4339,94.666728
1,3.574278,166.2446,84.909439
2,3.819722,177.9629,90.891311
3,3.839361,183.0995,93.469431
4,3.773028,178.9315,91.352264
5,3.640639,170.4126,87.026619
6,3.773028,178.9315,91.352264
7,3.817194,179.1058,91.461497
8,4.125861,197.2549,100.690381
9,3.759389,176.8699,90.314644


## Genetic Algorithm implementation

In [11]:
def initialize_population(population_size):
    population = generate_random_sequences(population_size,10)
    return population


In [12]:
def tournament_selection(population, fitness_values, tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmin(tournament_fitness)]
        selected_parents.append(population[winner_index])
    return selected_parents

In [13]:
import random

def ordered_crossover(parent1, parent2):
    length = len(parent1)
    crossover_point = random.randint(1, length - 2)
    

    while True:
        child = parent1[:crossover_point] + parent2[crossover_point:]
        if sequenceIsValid(child):
            break
        crossover_point = random.randint(1, length - 2)
        

    # print("Crossing : ", parent1, parent2)
    # print("Result     : ", child)

    return child


In [14]:
def shift_mutation(sequence):
    mutation_point1 = np.random.randint(1, len(sequence) - 2)
    mutation_point2 = np.random.randint(mutation_point1 + 1, len(sequence) - 1)
    mutated_sequence = sequence[:mutation_point1] + sequence[mutation_point2:] + sequence[mutation_point1:mutation_point2]
    mutated_sequence[0] = sequence[0]
    mutated_sequence[-1] = sequence[-1]

    print( " Original : ",sequence, " Mutated : ", mutated_sequence)
    return mutated_sequence

In [15]:
import random

def genetic_algorithm(num_generations, population_size, tournament_size, mutation_probability):
    population = initialize_population(population_size)
    
    for generation in range(num_generations):
        fitness_values = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0] for sequence in population]

        parents = tournament_selection(population, fitness_values, tournament_size)
        offspring = []

        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                child1 = ordered_crossover(parents[i], parents[i + 1])
                child2 = ordered_crossover(parents[i + 1], parents[i])
                offspring.extend([child1, child2])

        mutated_offspring = []
        for child in offspring:
            if random.uniform(0, 1) < mutation_probability:
                mutated_child = shift_mutation(child)
                mutated_offspring.append(mutated_child)
            else:
                mutated_offspring.append(child)

        population = mutated_offspring

    best_sequence = min(population, key=lambda x: calculate_cost(x, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0])

    return best_sequence


In [16]:
best_sequence = genetic_algorithm(num_generations=100, population_size=50, tournament_size=5, mutation_probability=0.0001)
print("Best Sequence:", best_sequence)
print("Best Cost:", calculate_cost(best_sequence, time_matrix, distance_matrix, w1, w2))

 Original :  [0, 6, 4, 8, 4, 3, 7, 8, 4, 5, 13, 16]  Mutated :  [0, 6, 4, 8, 4, 7, 8, 4, 5, 13, 16, 16]
 Original :  [0, 6, 4, 8, 4, 7, 8, 8, 4, 13, 16, 16]  Mutated :  [0, 6, 4, 8, 4, 7, 13, 16, 16, 8, 8, 16]
Best Sequence: [0, 6, 4, 8, 4, 7, 8, 8, 4, 13, 16, 16]
Best Cost:          Σt        Σd  w1*Σt+w2*Σd
0  2.256306  102.6526    52.454453
